# 確率モデル

## データの読み込み

In [1]:
import io

path = "data.txt"
with io.open(path, encoding="utf-8") as f:
    data = f.read().split()
    
data

['100名まで収容可能な会場。',
 'ドレスのご試着は、',
 'ご要望にお応えします。',
 '写真撮影を行います。',
 '宜しくお願い致します。',
 '私は犬を散歩する。']

## 前処理 (分かち書き)

In [2]:
from janome.tokenizer import Tokenizer

t = Tokenizer()
def wakati(text):
    w = t.tokenize(text, wakati=True)
    return " ".join(w)

data = [wakati(w) for w in data]

data

['100 名 まで 収容 可能 な 会場 。',
 'ドレス の ご 試着 は 、',
 'ご 要望 に お 応え し ます 。',
 '写真 撮影 を 行い ます 。',
 '宜しく お願い 致し ます 。',
 '私 は 犬 を 散歩 する 。']

KerasのTokenizerでベクトル化を行う前に, JanomeのTokenizerで分かち書きの前処理を行う. 

## ライブラリの読み込み

In [3]:
from tensorflow import keras
from keras.preprocessing import sequence
from keras import preprocessing
import keras
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional

/Users/h-tanaka/opt/anaconda3/lib/python3.7/site-packages/tensorflow-1.14.0-py3.7-macosx-10.9-x86_64.egg/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/h-tanaka/opt/anaconda3/lib/python3.7/site-packages/tensorflow-1.14.0-py3.7-macosx-10.9-x86_64.egg/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/h-tanaka/opt/anaconda3/lib/python3.7/site-packages/tensorflow-1.14.0-py3.7-macosx-10.9-x86_64.egg/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood a

## 前処理 (ベクトル化)

In [4]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data)
vocab = tokenizer.word_index
seqs = tokenizer.texts_to_sequences(data)

In [5]:
vocab

{'。': 1,
 'ます': 2,
 'ご': 3,
 'は': 4,
 'を': 5,
 '100': 6,
 '名': 7,
 'まで': 8,
 '収容': 9,
 '可能': 10,
 'な': 11,
 '会場': 12,
 'ドレス': 13,
 'の': 14,
 '試着': 15,
 '、': 16,
 '要望': 17,
 'に': 18,
 'お': 19,
 '応え': 20,
 'し': 21,
 '写真': 22,
 '撮影': 23,
 '行い': 24,
 '宜しく': 25,
 'お願い': 26,
 '致し': 27,
 '私': 28,
 '犬': 29,
 '散歩': 30,
 'する': 31}

In [6]:
seqs

[[6, 7, 8, 9, 10, 11, 12, 1],
 [13, 14, 3, 15, 4, 16],
 [3, 17, 18, 19, 20, 21, 2, 1],
 [22, 23, 5, 24, 2, 1],
 [25, 26, 27, 2, 1],
 [28, 4, 29, 5, 30, 31, 1]]

## シーケンスを同じ長さになるように詰める

In [7]:
def prepare_sentence(seq, maxlen):
    x = []
    y = []
    for i, w in enumerate(seq):
        x_padded = pad_sequences([seq[:i]],
                                 maxlen=maxlen - 1,
                                 padding='pre')[0]
        x.append(x_padded)
        y.append(w)
    return x, y

## xとyを準備する

In [8]:
maxlen = max([len(seq) for seq in seqs])
x = []
y = []
for seq in seqs:
    x_windows, y_windows = prepare_sentence(seq, maxlen)
    x += x_windows
    y += y_windows
x = np.array(x)
y = np.array(y) - 1
y = np.eye(len(vocab))[y]

## モデリング

In [9]:
model = Sequential()
model.add(Embedding(input_dim=len(vocab) + 1,  
                    output_dim=5,
                    input_length=maxlen - 1)) 
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.5))
model.add(Dense(len(vocab), activation='softmax'))

model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

## 学習させる

In [10]:
model.fit(x, y, epochs=1000)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/1000
40/40 [==============================] - 1s 21ms/step - loss: 0.1425 - accuracy: 0.9677
Epoch 2/1000
40/40 [==============================] - 0s 657us/step - loss: 0.1425 - accuracy: 0.9677
Epoch 3/1000
40/40 [==============================] - 0s 713us/step - loss: 0.1424 - accuracy: 0.9677
Epoch 4/1000
40/40 [==============================] - 0s 700us/step - loss: 0.1423 - accuracy: 0.9677
Epoch 5/1000
40/40 [==============================] - 0s 808us/step - loss: 0.1423 - accuracy: 0.9677
Epoch 6/1000
40/40 [==============================] - 0s 747us/step - loss: 0.1422 - accuracy: 0.9677
Epoch 7/1000
40/40 [==============================] - 0s 738us/step - loss: 0.1422 - accuracy: 0.9677
Epoch 8/1000
40/40 [==============================] - 0s 618us/step - loss: 0.1421 - accuracy: 0.9677
Epoch 9/1000
40/40 [==============================] - 0s 776us/step - loss: 0.1420 - accur

40/40 [==============================] - 0s 657us/step - loss: 0.1135 - accuracy: 0.9694
Epoch 75/1000
40/40 [==============================] - 0s 623us/step - loss: 0.1111 - accuracy: 0.9702
Epoch 76/1000
40/40 [==============================] - 0s 701us/step - loss: 0.1099 - accuracy: 0.9702
Epoch 77/1000
40/40 [==============================] - 0s 678us/step - loss: 0.1092 - accuracy: 0.9702
Epoch 78/1000
40/40 [==============================] - 0s 607us/step - loss: 0.1073 - accuracy: 0.9710
Epoch 79/1000
40/40 [==============================] - 0s 626us/step - loss: 0.1093 - accuracy: 0.9694
Epoch 80/1000
40/40 [==============================] - 0s 642us/step - loss: 0.1093 - accuracy: 0.9702
Epoch 81/1000
40/40 [==============================] - 0s 647us/step - loss: 0.1051 - accuracy: 0.9710
Epoch 82/1000
40/40 [==============================] - 0s 629us/step - loss: 0.1088 - accuracy: 0.9694
Epoch 83/1000
40/40 [==============================] - 0s 618us/step - loss: 0.1067 - a

40/40 [==============================] - 0s 642us/step - loss: 0.0834 - accuracy: 0.9726
Epoch 153/1000
40/40 [==============================] - 0s 711us/step - loss: 0.0761 - accuracy: 0.9750
Epoch 154/1000
40/40 [==============================] - 0s 670us/step - loss: 0.0755 - accuracy: 0.9750
Epoch 155/1000
40/40 [==============================] - 0s 596us/step - loss: 0.0754 - accuracy: 0.9758
Epoch 156/1000
40/40 [==============================] - 0s 765us/step - loss: 0.0767 - accuracy: 0.9758
Epoch 157/1000
40/40 [==============================] - 0s 746us/step - loss: 0.0763 - accuracy: 0.9742
Epoch 158/1000
40/40 [==============================] - 0s 687us/step - loss: 0.0790 - accuracy: 0.9726
Epoch 159/1000
40/40 [==============================] - 0s 668us/step - loss: 0.0792 - accuracy: 0.9758
Epoch 160/1000
40/40 [==============================] - 0s 727us/step - loss: 0.0774 - accuracy: 0.9734
Epoch 161/1000
40/40 [==============================] - 0s 696us/step - loss: 0

40/40 [==============================] - 0s 727us/step - loss: 0.0478 - accuracy: 0.9839
Epoch 231/1000
40/40 [==============================] - 0s 739us/step - loss: 0.0505 - accuracy: 0.9815
Epoch 232/1000
40/40 [==============================] - 0s 751us/step - loss: 0.0513 - accuracy: 0.9839
Epoch 233/1000
40/40 [==============================] - 0s 616us/step - loss: 0.0494 - accuracy: 0.9847
Epoch 234/1000
40/40 [==============================] - 0s 730us/step - loss: 0.0490 - accuracy: 0.9831
Epoch 235/1000
40/40 [==============================] - 0s 733us/step - loss: 0.0490 - accuracy: 0.9839
Epoch 236/1000
40/40 [==============================] - 0s 754us/step - loss: 0.0447 - accuracy: 0.9855
Epoch 237/1000
40/40 [==============================] - 0s 720us/step - loss: 0.0492 - accuracy: 0.9847
Epoch 238/1000
40/40 [==============================] - 0s 756us/step - loss: 0.0448 - accuracy: 0.9847
Epoch 239/1000
40/40 [==============================] - 0s 730us/step - loss: 0

Epoch 309/1000
40/40 [==============================] - 0s 789us/step - loss: 0.0283 - accuracy: 0.9911
Epoch 310/1000
40/40 [==============================] - 0s 837us/step - loss: 0.0287 - accuracy: 0.9919
Epoch 311/1000
40/40 [==============================] - 0s 740us/step - loss: 0.0288 - accuracy: 0.9919
Epoch 312/1000
40/40 [==============================] - 0s 702us/step - loss: 0.0301 - accuracy: 0.9895
Epoch 313/1000
40/40 [==============================] - 0s 819us/step - loss: 0.0309 - accuracy: 0.9911
Epoch 314/1000
40/40 [==============================] - 0s 728us/step - loss: 0.0288 - accuracy: 0.9911
Epoch 315/1000
40/40 [==============================] - 0s 673us/step - loss: 0.0314 - accuracy: 0.9903
Epoch 316/1000
40/40 [==============================] - 0s 745us/step - loss: 0.0318 - accuracy: 0.9895
Epoch 317/1000
40/40 [==============================] - 0s 749us/step - loss: 0.0349 - accuracy: 0.9879
Epoch 318/1000
40/40 [==============================] - 0s 763us

...

40/40 [==============================] - 0s 777us/step - loss: 0.0166 - accuracy: 0.9952
Epoch 623/1000
40/40 [==============================] - 0s 967us/step - loss: 0.0179 - accuracy: 0.9952
Epoch 624/1000
40/40 [==============================] - 0s 846us/step - loss: 0.0190 - accuracy: 0.9952
Epoch 625/1000
40/40 [==============================] - 0s 838us/step - loss: 0.0200 - accuracy: 0.9935
Epoch 626/1000
40/40 [==============================] - 0s 1ms/step - loss: 0.0193 - accuracy: 0.9944
Epoch 627/1000
40/40 [==============================] - 0s 755us/step - loss: 0.0172 - accuracy: 0.9952
Epoch 628/1000
40/40 [==============================] - 0s 743us/step - loss: 0.0165 - accuracy: 0.9952
Epoch 629/1000
40/40 [==============================] - 0s 938us/step - loss: 0.0168 - accuracy: 0.9952
Epoch 630/1000
40/40 [==============================] - 0s 716us/step - loss: 0.0183 - accuracy: 0.9944
Epoch 631/1000
40/40 [==============================] - 0s 865us/step - loss: 0.0

40/40 [==============================] - 0s 958us/step - loss: 0.0175 - accuracy: 0.9952
Epoch 702/1000
40/40 [==============================] - 0s 772us/step - loss: 0.0188 - accuracy: 0.9944
Epoch 703/1000
40/40 [==============================] - 0s 839us/step - loss: 0.0175 - accuracy: 0.9952
Epoch 704/1000
40/40 [==============================] - 0s 791us/step - loss: 0.0172 - accuracy: 0.9944
Epoch 705/1000
40/40 [==============================] - 0s 1ms/step - loss: 0.0152 - accuracy: 0.9944
Epoch 706/1000
40/40 [==============================] - 0s 757us/step - loss: 0.0173 - accuracy: 0.9952
Epoch 707/1000
40/40 [==============================] - 0s 887us/step - loss: 0.0162 - accuracy: 0.9952
Epoch 708/1000
40/40 [==============================] - 0s 850us/step - loss: 0.0160 - accuracy: 0.9952
Epoch 709/1000
40/40 [==============================] - 0s 861us/step - loss: 0.0134 - accuracy: 0.9952
Epoch 710/1000
40/40 [==============================] - 0s 896us/step - loss: 0.0

Epoch 780/1000
40/40 [==============================] - 0s 731us/step - loss: 0.0172 - accuracy: 0.9952
Epoch 781/1000
40/40 [==============================] - 0s 760us/step - loss: 0.0182 - accuracy: 0.9935
Epoch 782/1000
40/40 [==============================] - 0s 768us/step - loss: 0.0165 - accuracy: 0.9952
Epoch 783/1000
40/40 [==============================] - 0s 797us/step - loss: 0.0154 - accuracy: 0.9952
Epoch 784/1000
40/40 [==============================] - 0s 1ms/step - loss: 0.0173 - accuracy: 0.9952
Epoch 785/1000
40/40 [==============================] - 0s 714us/step - loss: 0.0175 - accuracy: 0.9952
Epoch 786/1000
40/40 [==============================] - 0s 788us/step - loss: 0.0140 - accuracy: 0.9952
Epoch 787/1000
40/40 [==============================] - 0s 751us/step - loss: 0.0166 - accuracy: 0.9952
Epoch 788/1000
40/40 [==============================] - 0s 772us/step - loss: 0.0170 - accuracy: 0.9944
Epoch 789/1000
40/40 [==============================] - 0s 787us/s

40/40 [==============================] - 0s 715us/step - loss: 0.0154 - accuracy: 0.9952
Epoch 859/1000
40/40 [==============================] - 0s 735us/step - loss: 0.0136 - accuracy: 0.9952
Epoch 860/1000
40/40 [==============================] - 0s 790us/step - loss: 0.0153 - accuracy: 0.9952
Epoch 861/1000
40/40 [==============================] - 0s 760us/step - loss: 0.0159 - accuracy: 0.9952
Epoch 862/1000
40/40 [==============================] - 0s 708us/step - loss: 0.0161 - accuracy: 0.9952
Epoch 863/1000
40/40 [==============================] - 0s 702us/step - loss: 0.0159 - accuracy: 0.9952
Epoch 864/1000
40/40 [==============================] - 0s 676us/step - loss: 0.0163 - accuracy: 0.9952
Epoch 865/1000
40/40 [==============================] - 0s 758us/step - loss: 0.0142 - accuracy: 0.9952
Epoch 866/1000
40/40 [==============================] - 0s 738us/step - loss: 0.0154 - accuracy: 0.9952
Epoch 867/1000
40/40 [==============================] - 0s 721us/step - loss: 0

40/40 [==============================] - 0s 717us/step - loss: 0.0147 - accuracy: 0.9952
Epoch 937/1000
40/40 [==============================] - 0s 619us/step - loss: 0.0183 - accuracy: 0.9952
Epoch 938/1000
40/40 [==============================] - 0s 691us/step - loss: 0.0147 - accuracy: 0.9952
Epoch 939/1000
40/40 [==============================] - 0s 634us/step - loss: 0.0159 - accuracy: 0.9952
Epoch 940/1000
40/40 [==============================] - 0s 614us/step - loss: 0.0164 - accuracy: 0.9952
Epoch 941/1000
40/40 [==============================] - 0s 679us/step - loss: 0.0171 - accuracy: 0.9952
Epoch 942/1000
40/40 [==============================] - 0s 612us/step - loss: 0.0150 - accuracy: 0.9952
Epoch 943/1000
40/40 [==============================] - 0s 659us/step - loss: 0.0168 - accuracy: 0.9952
Epoch 944/1000
40/40 [==============================] - 0s 642us/step - loss: 0.0128 - accuracy: 0.9952
Epoch 945/1000
40/40 [==============================] - 0s 677us/step - loss: 0

## 発生確率を計算する

In [11]:
input_sentence = "私は犬に散歩する。"
sentence = t.tokenize(input_sentence, wakati=True)
tok = tokenizer.texts_to_sequences([sentence])[0]
x_test, y_test = prepare_sentence(tok, maxlen)
x_test = np.array(x_test)
y_test = np.array(y_test) - 1 
p_pred = model.predict(x_test)  
vocab_inv = {v: k for k, v in vocab.items()}

・ input_sentenceに誤字脱字, 衍字をチェックしたい文章を入力する. <br>
・ ある単語の次に来る単語が正しいか, 誤っているかを確率的に判断する.

## 算出した確率をもとに正常か, 異常かを判定する

In [13]:
log_p_sentence = 0
err = []
words = []
for i, prob in enumerate(p_pred):
    word = vocab_inv[y_test[i]+1]
    words.append(word)
    history = ' '.join([vocab_inv[w] for w in x_test[i, :] if w != 0])
    prob_word = prob[y_test[i]]
    log_p_sentence += np.log(prob_word)
    
    if prob_word < 0.01:
        err.append(word)

    print('P(w={}|h={})={}'.format(word, history, prob_word))
print('Prob. sentence: {}'.format(np.exp(log_p_sentence)))

# 「誤字脱字箇所」と「誤字脱字を含む文」を出力.
if len(err) != 0:
    print("NG : " + str(err))
    print(input_sentence)
    
# 訂正箇所なしの場合
else:
    print("OK")

P(w=私|h=)=0.16551995277404785
P(w=は|h=私)=0.9951953291893005
P(w=犬|h=私 は)=0.9960660338401794
P(w=に|h=私 は 犬)=2.611126546980813e-06
P(w=散歩|h=私 は 犬 に)=0.9933903217315674
P(w=する|h=私 は 犬 に 散歩)=0.9988625049591064
P(w=。|h=私 は 犬 に 散歩 する)=0.9998970031738281
Prob. sentence: 4.250651700854218e-07
NG : ['に']
私は犬に散歩する。


&nbsp;

# 言語モデル

## データの読み込み

In [14]:
from janome.tokenizer import Tokenizer

path = "data.txt"
file = open(path, "r", encoding="utf-8")
text = file.read()

text

'100名まで収容可能な会場。\nドレスのご試着は、\nご要望にお応えします。\n写真撮影を行います。\n宜しくお願い致します。\n私は犬を散歩する。'

## 前処理 (分かち書き)

In [15]:
from janome.tokenizer import Tokenizer

t = Tokenizer()
def wakati(text):
    w = t.tokenize(text, wakati=True)
    return " ".join(w)

data = [wakati(w) for w in text]

data

['1',
 '0',
 '0',
 '名',
 'ま',
 'で',
 '収',
 '容',
 '可',
 '能',
 'な',
 '会',
 '場',
 '。',
 '',
 'ド',
 'レ',
 'ス',
 'の',
 'ご',
 '試',
 '着',
 'は',
 '、',
 '',
 'ご',
 '要',
 '望',
 'に',
 'お',
 '応',
 'え',
 'し',
 'ま',
 'す',
 '。',
 '',
 '写',
 '真',
 '撮',
 '影',
 'を',
 '行',
 'い',
 'ま',
 'す',
 '。',
 '',
 '宜',
 'し',
 'く',
 'お',
 '願',
 'い',
 '致',
 'し',
 'ま',
 'す',
 '。',
 '',
 '私',
 'は',
 '犬',
 'を',
 '散',
 '歩',
 'す',
 'る',
 '。']

## モデルから文字列を生成

In [16]:
def generate_seq(model, tokenizer, seed_text, n_words):
    in_text, result = seed_text, seed_text
    for _ in range(n_words):
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        encoded = np.array(encoded)
        # 学習データから文字列を予測。
        yhat = model.predict_classes(encoded, verbose=0)
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        in_text, result = out_word, result + ' ' + out_word
    return result

## 前処理 (ベクトル化)

In [17]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
encoded = tokenizer.texts_to_sequences([data])[0]

## 語彙のサイズの決定

In [18]:
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)

sequences = list()
for i in range(1, len(encoded)):
    sequence = encoded[i-1:i+1]
    sequences.append(sequence)
print('Total Sequences: %d' % len(sequences))

Vocabulary Size: 48
Total Sequences: 68


## Xとy要素に分割する

In [19]:
sequences = np.array(sequences)
X, y = sequences[:,0],sequences[:,1]

## One-hotエンコーディング

In [20]:
y = to_categorical(y, num_classes=vocab_size)

## モデリング

In [21]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=1))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.5))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1, 10)             480       
_________________________________________________________________
bidirectional_2 (Bidirection (None, 128)               38400     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 48)                6192      
Total params: 45,072
Trainable params: 45,072
Non-trainable params: 0
_________________________________________________________________
None


## 学習させる

In [22]:
model.fit(X, y, epochs=500, verbose=2)

Epoch 1/500
 - 1s - loss: 3.8720 - accuracy: 0.0147
Epoch 2/500
 - 0s - loss: 3.8692 - accuracy: 0.0441
Epoch 3/500
 - 0s - loss: 3.8677 - accuracy: 0.0735
Epoch 4/500
 - 0s - loss: 3.8663 - accuracy: 0.1471
Epoch 5/500
 - 0s - loss: 3.8639 - accuracy: 0.1324
Epoch 6/500
 - 0s - loss: 3.8630 - accuracy: 0.1176
Epoch 7/500
 - 0s - loss: 3.8612 - accuracy: 0.1176
Epoch 8/500
 - 0s - loss: 3.8598 - accuracy: 0.1324
Epoch 9/500
 - 0s - loss: 3.8579 - accuracy: 0.1324
Epoch 10/500
 - 0s - loss: 3.8565 - accuracy: 0.1471
Epoch 11/500
 - 0s - loss: 3.8546 - accuracy: 0.1618
Epoch 12/500
 - 0s - loss: 3.8524 - accuracy: 0.1765
Epoch 13/500
 - 0s - loss: 3.8506 - accuracy: 0.1471
Epoch 14/500
 - 0s - loss: 3.8481 - accuracy: 0.1765
Epoch 15/500
 - 0s - loss: 3.8478 - accuracy: 0.1765
Epoch 16/500
 - 0s - loss: 3.8444 - accuracy: 0.1324
Epoch 17/500
 - 0s - loss: 3.8417 - accuracy: 0.1471
Epoch 18/500
 - 0s - loss: 3.8386 - accuracy: 0.1765
Epoch 19/500
 - 0s - loss: 3.8376 - accuracy: 0.1618
Ep

Epoch 155/500
 - 0s - loss: 1.4694 - accuracy: 0.6912
Epoch 156/500
 - 0s - loss: 1.4584 - accuracy: 0.6765
Epoch 157/500
 - 0s - loss: 1.4028 - accuracy: 0.7206
Epoch 158/500
 - 0s - loss: 1.3829 - accuracy: 0.6324
Epoch 159/500
 - 0s - loss: 1.3722 - accuracy: 0.7353
Epoch 160/500
 - 0s - loss: 1.3799 - accuracy: 0.7647
Epoch 161/500
 - 0s - loss: 1.3657 - accuracy: 0.7500
Epoch 162/500
 - 0s - loss: 1.4185 - accuracy: 0.6471
Epoch 163/500
 - 0s - loss: 1.3523 - accuracy: 0.7647
Epoch 164/500
 - 0s - loss: 1.3216 - accuracy: 0.7353
Epoch 165/500
 - 0s - loss: 1.3009 - accuracy: 0.7647
Epoch 166/500
 - 0s - loss: 1.3236 - accuracy: 0.7353
Epoch 167/500
 - 0s - loss: 1.3263 - accuracy: 0.7647
Epoch 168/500
 - 0s - loss: 1.2890 - accuracy: 0.7647
Epoch 169/500
 - 0s - loss: 1.2867 - accuracy: 0.7500
Epoch 170/500
 - 0s - loss: 1.2588 - accuracy: 0.7794
Epoch 171/500
 - 0s - loss: 1.2485 - accuracy: 0.7500
Epoch 172/500
 - 0s - loss: 1.2791 - accuracy: 0.7500
Epoch 173/500
 - 0s - loss: 

Epoch 307/500
 - 0s - loss: 0.5191 - accuracy: 0.8235
Epoch 308/500
 - 0s - loss: 0.5321 - accuracy: 0.7794
Epoch 309/500
 - 0s - loss: 0.5294 - accuracy: 0.8088
Epoch 310/500
 - 0s - loss: 0.4974 - accuracy: 0.7941
Epoch 311/500
 - 0s - loss: 0.4674 - accuracy: 0.8235
Epoch 312/500
 - 0s - loss: 0.5001 - accuracy: 0.8088
Epoch 313/500
 - 0s - loss: 0.5045 - accuracy: 0.8235
Epoch 314/500
 - 0s - loss: 0.4827 - accuracy: 0.7794
Epoch 315/500
 - 0s - loss: 0.4760 - accuracy: 0.8235
Epoch 316/500
 - 0s - loss: 0.4679 - accuracy: 0.8235
Epoch 317/500
 - 0s - loss: 0.4720 - accuracy: 0.8382
Epoch 318/500
 - 0s - loss: 0.5049 - accuracy: 0.8088
Epoch 319/500
 - 0s - loss: 0.4738 - accuracy: 0.8235
Epoch 320/500
 - 0s - loss: 0.5208 - accuracy: 0.7794
Epoch 321/500
 - 0s - loss: 0.5239 - accuracy: 0.7794
Epoch 322/500
 - 0s - loss: 0.4395 - accuracy: 0.8529
Epoch 323/500
 - 0s - loss: 0.4742 - accuracy: 0.8235
Epoch 324/500
 - 0s - loss: 0.4881 - accuracy: 0.7941
Epoch 325/500
 - 0s - loss: 

Epoch 459/500
 - 0s - loss: 0.4220 - accuracy: 0.7500
Epoch 460/500
 - 0s - loss: 0.3594 - accuracy: 0.8529
Epoch 461/500
 - 0s - loss: 0.4042 - accuracy: 0.7941
Epoch 462/500
 - 0s - loss: 0.3776 - accuracy: 0.8382
Epoch 463/500
 - 0s - loss: 0.3931 - accuracy: 0.7941
Epoch 464/500
 - 0s - loss: 0.4318 - accuracy: 0.7794
Epoch 465/500
 - 0s - loss: 0.4031 - accuracy: 0.8235
Epoch 466/500
 - 0s - loss: 0.3604 - accuracy: 0.8235
Epoch 467/500
 - 0s - loss: 0.3557 - accuracy: 0.8824
Epoch 468/500
 - 0s - loss: 0.4209 - accuracy: 0.7794
Epoch 469/500
 - 0s - loss: 0.4152 - accuracy: 0.8088
Epoch 470/500
 - 0s - loss: 0.3995 - accuracy: 0.8235
Epoch 471/500
 - 0s - loss: 0.4136 - accuracy: 0.7794
Epoch 472/500
 - 0s - loss: 0.4211 - accuracy: 0.7794
Epoch 473/500
 - 0s - loss: 0.4156 - accuracy: 0.8235
Epoch 474/500
 - 0s - loss: 0.3790 - accuracy: 0.8235
Epoch 475/500
 - 0s - loss: 0.3955 - accuracy: 0.8088
Epoch 476/500
 - 0s - loss: 0.4011 - accuracy: 0.7941
Epoch 477/500
 - 0s - loss: 

## 生成開始地点を決定

In [23]:
typo_words = []
typo_index = []
if len(words) != len(sentence):
    for i in range(len(sentence)):
        if not str(sentence[i]) in words:
            typo_words.append(str(sentence[i]))
            typo_index.append(sentence.index(typo_words[0]))
            
else:
    for i in range(len(err)):
        for j in range(len(sentence)):
            if str(err[i]) == str(sentence[j]):
                typo_words.append(str(err[i]))
                typo_index.append(sentence.index(typo_words[0]))

search_index = []
search_word = []
for i in range(len(typo_index)):
    search_index.append(int(typo_index[i])-1)
    search_word.append(sentence[search_index[i]])

## 評価

In [24]:
# 評価
for i in range(len(search_word)):
    print("prediction: " + generate_seq(model, tokenizer, str(search_word[i]) , 1))

prediction: 犬 を
